In [3]:
! pip install git+https://github.com/unifyai/models.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/unifyai/models.git to /tmp/pip-req-build-g3tw_5ug
  Running command git clone --filter=blob:none --quiet https://github.com/unifyai/models.git /tmp/pip-req-build-g3tw_5ug
  Resolved https://github.com/unifyai/models.git to commit cb97624090d8d44ca3abb107cb7e502ba1a435fd
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


# layers

In [12]:
import ivy


class InceptionConvBlock(ivy.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(InceptionConvBlock, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = ivy.Conv2D(
            self.in_channels,
            self.out_channels,
            self.kernel_size,
            self.stride,
            self.padding,
            with_bias=False,
            data_format="NCHW",
        )
        self.bn = ivy.BatchNorm2D(self.out_channels, eps=0.001, data_format="NCS")

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = ivy.relu(x)
        return x


class InceptionBlock(ivy.Module):
    def __init__(
        self,
        in_channels,
        num1x1,
        num3x3_reduce,
        num3x3,
        num5x5_reduce,
        num5x5,
        pool_proj,
    ):
        self.in_channels = in_channels
        self.num1x1 = num1x1
        self.num3x3_reduce = num3x3_reduce
        self.num3x3 = num3x3
        self.num5x5_reduce = num5x5_reduce
        self.num5x5 = num5x5
        self.pool_proj = pool_proj
        super(InceptionBlock, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv_1x1 = InceptionConvBlock(
            self.in_channels, self.num1x1, kernel_size=[1, 1], stride=1, padding=0
        )

        self.conv_3x3 = InceptionConvBlock(
            self.in_channels,
            self.num3x3_reduce,
            kernel_size=[1, 1],
            stride=1,
            padding=0,
        )
        self.conv_3x3_red = InceptionConvBlock(
            self.num3x3_reduce, self.num3x3, kernel_size=[3, 3], stride=1, padding=1
        )

        self.conv_5x5 = InceptionConvBlock(
            self.in_channels,
            self.num5x5_reduce,
            kernel_size=[1, 1],
            stride=1,
            padding=0,
        )
        self.conv_5x5_red = InceptionConvBlock(
            self.num5x5_reduce, self.num5x5, kernel_size=[3, 3], stride=1, padding=1
        )

        self.pool_proj_conv = InceptionConvBlock(
            self.in_channels, self.pool_proj, kernel_size=[1, 1], stride=1, padding=0
        )

    def _forward(self, x):
        # 1x1
        conv_1x1 = self.conv_1x1(x)

        # 3x3
        conv_3x3 = self.conv_3x3(x)
        conv_3x3_red = self.conv_3x3_red(conv_3x3)

        # 5x5
        conv_5x5 = self.conv_5x5(x)
        conv_5x5_red = self.conv_5x5_red(conv_5x5)

        # pool_proj
        pool_proj = ivy.max_pool2d(x, [3, 3], 1, 1, ceil_mode=True, data_format="NCHW")
        pool_proj = self.pool_proj_conv(pool_proj)

        ret = ivy.concat([conv_1x1, conv_3x3_red, conv_5x5_red, pool_proj], axis=1)
        return ret


class InceptionAuxiliaryBlock(ivy.Module):
    def __init__(self, in_channels, num_classes, aux_dropout=0.7):
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.aux_dropout = aux_dropout
        super(InceptionAuxiliaryBlock, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = InceptionConvBlock(self.in_channels, 128, [1, 1], 1, 0)
        self.fc1 = ivy.Linear(2048, 1024)
        self.dropout = ivy.Dropout(self.aux_dropout)
        self.fc2 = ivy.Linear(1024, self.num_classes)
        self.softmax = ivy.Softmax()

    def _forward(self, x):
        out = ivy.adaptive_avg_pool2d(x, [4, 4])
        out = self.conv(out)
        out = ivy.flatten(out, start_dim=1)
        out = self.fc1(out)
        out = ivy.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out


# googlenet

In [9]:
import torch

def load_torch_weights(
    url,
    ref_model,
    raw_keys_to_prune=[],
    ref_keys_to_prune=[],
    custom_mapping=None,
    map_location=torch.device("cpu"),
):
    ivy_torch = ivy.with_backend("torch")
    weights = torch.hub.load_state_dict_from_url(url, map_location=map_location)
    display(ivy.asarray(ivy.Container(weights)))

    weights_raw = ivy.Container(
        ivy_torch.to_numpy(ivy_torch.Container(weights)).cont_to_dict()
    )
    weights_raw, weights_ref, pruned_ref = _prune_keys(
        weights_raw, ref_model.v, raw_keys_to_prune, ref_keys_to_prune
    )
    mapping = _map_weights(weights_raw, weights_ref, custom_mapping=custom_mapping)
    w_clean = weights_raw.cont_restructure(mapping, keep_orig=False)
    if ref_keys_to_prune:
        w_clean = ivy.Container.cont_combine(w_clean, pruned_ref)
    return ivy.asarray(w_clean)


In [10]:

def _prune_keys(raw, ref, raw_keys_to_prune=[], ref_keys_to_prune=[]):
    pruned_ref = {}
    if raw_keys_to_prune:
        raw = raw.cont_prune_keys(raw_keys_to_prune)
    if ref_keys_to_prune:
        pruned_ref = ref.cont_at_keys(ref_keys_to_prune)
        ref = ref.cont_prune_keys(ref_keys_to_prune)
    return raw, ref, pruned_ref


def _map_weights(raw, ref, custom_mapping=None):
    mapping = {}
    for old_key, new_key in zip(
        raw.cont_sort_by_key().cont_to_iterator_keys(),
        ref.cont_sort_by_key().cont_to_iterator_keys(),
    ):
        new_mapping = new_key
        if custom_mapping is not None:
            new_mapping = custom_mapping(old_key, new_key)
        mapping[old_key] = new_mapping
    return mapping


In [11]:
# global
import ivy
import ivy_models
from ivy_models.base import BaseSpec, BaseModel
# from ivy_models.googlenet.layers import (
#     InceptionConvBlock,
#     InceptionBlock,
#     InceptionAuxiliaryBlock,
# )


class GoogLeNetSpec(BaseSpec):
    def __init__(
        self,
        training=False,
        num_classes=1000,
        dropout=0.4,
        aux_dropout=0.7,
        data_format="NCHW",
    ):
        if not training:
            dropout = 0
            aux_dropout = 0
        super(GoogLeNetSpec, self).__init__(
            training=training,
            num_classes=num_classes,
            dropout=dropout,
            aux_dropout=aux_dropout,
            data_format=data_format,
        )


class GoogLeNet(BaseModel):
    def __init__(
        self,
        training=False,
        num_classes=1000,
        dropout=0.4,
        aux_dropout=0.7,
        data_format="NCHW",
        spec=None,
        v=None,
    ):
        self.spec = (
            spec
            if spec and isinstance(spec, GoogLeNetSpec)
            else GoogLeNetSpec(
                training=training,
                num_classes=num_classes,
                dropout=dropout,
                aux_dropout=aux_dropout,
                data_format=data_format,
            )
        )
        super(GoogLeNet, self).__init__(v=v)

    def _build(self, *args, **kwargs):
        self.conv1 = InceptionConvBlock(3, 64, [7, 7], 2, padding=3)

        self.conv2 = InceptionConvBlock(64, 64, [1, 1], 1, padding=0)
        self.conv3 = InceptionConvBlock(64, 192, [3, 3], 1, padding=1)

        self.inception3A = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3B = InceptionBlock(256, 128, 128, 192, 32, 96, 64)

        self.inception4A = InceptionBlock(480, 192, 96, 208, 16, 48, 64)

        self.aux4A = InceptionAuxiliaryBlock(
            512, self.spec.num_classes, self.spec.aux_dropout
        )

        self.inception4B = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4C = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4D = InceptionBlock(512, 112, 144, 288, 32, 64, 64)

        self.aux4D = InceptionAuxiliaryBlock(
            528, self.spec.num_classes, self.spec.aux_dropout
        )

        self.inception4E = InceptionBlock(528, 256, 160, 320, 32, 128, 128)

        self.inception5A = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5B = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        self.pool6 = ivy.AdaptiveAvgPool2d([1, 1])

        self.dropout = ivy.Dropout(self.spec.dropout)
        self.fc = ivy.Linear(1024, self.spec.num_classes, with_bias=False)

    @classmethod
    def get_spec_class(self):
        return GoogLeNetSpec

    def _forward(self, x, data_format=None):
        data_format = data_format if data_format else self.spec.data_format
        if data_format == "NHWC":
            x = ivy.permute_dims(x, (0, 3, 1, 2))

        out = self.conv1(x)
        out = ivy.max_pool2d(out, [3, 3], 2, 0, ceil_mode=True, data_format="NCHW")
        out = self.conv2(out)
        out = self.conv3(out)
        out = ivy.max_pool2d(out, [3, 3], 2, 0, ceil_mode=True, data_format="NCHW")
        out = self.inception3A(out)
        out = self.inception3B(out)
        out = ivy.max_pool2d(out, [3, 3], 2, 0, ceil_mode=True, data_format="NCHW")
        out = self.inception4A(out)

        aux1 = None
        if self.spec.training:
            aux1 = self.aux4A(out)

        out = self.inception4B(out)
        out = self.inception4C(out)
        out = self.inception4D(out)

        aux2 = None
        if self.spec.training:
            aux2 = self.aux4D(out)

        out = self.inception4E(out)
        out = ivy.max_pool2d(out, [2, 2], 2, 0, ceil_mode=True, data_format="NCHW")
        out = self.inception5A(out)
        out = self.inception5B(out)
        out = self.pool6(out)
        out = ivy.flatten(out, start_dim=1)
        out = self.dropout(out)
        out = self.fc(out)
        return out, aux1, aux2


def _inceptionNet_torch_weights_mapping(old_key, new_key):
    if "conv/weight" in old_key:
        return {"key_chain": new_key, "pattern": "b c h w -> h w c b"}
    return new_key


def inceptionNet_v1(
    pretrained=True,
    training=False,
    num_classes=1000,
    dropout=0.4,
    aux_dropout=0.7,
    data_format="NCHW",
):
    """InceptionNet-V1 model"""
    model = GoogLeNet(
        training=training,
        num_classes=num_classes,
        dropout=dropout,
        aux_dropout=aux_dropout,
        data_format=data_format,
    )
    if pretrained:
        url = "https://download.pytorch.org/models/googlenet-1378be20.pth"
        w_clean = load_torch_weights(
            url,
            model,
            raw_keys_to_prune=["num_batches_tracked"],
            custom_mapping=_inceptionNet_torch_weights_mapping,
        )
        model.v = w_clean
    return model


# test

In [13]:
ivy.set_backend('torch')
inceptionNet_v1()

{
    aux1: {
        conv: {
            bn: {
                bias: (<class ivy.data_classes.array.array.Array> shape=[128]),
                num_batches_tracked: ivy.array(0),
                running_mean: (<class ivy.data_classes.array.array.Array> shape=[128]),
                running_var: (<class ivy.data_classes.array.array.Array> shape=[128]),
                weight: (<class ivy.data_classes.array.array.Array> shape=[128])
            },
            conv: {
                weight: (<class ivy.data_classes.array.array.Array> shape=[128, 512, 1, 1])
            }
        },
        fc1: {
            bias: (<class ivy.data_classes.array.array.Array> shape=[1024]),
            weight: (<class ivy.data_classes.array.array.Array> shape=[1024, 2048])
        },
        fc2: {
            bias: (<class ivy.data_classes.array.array.Array> shape=[1000]),
            weight: (<class ivy.data_classes.array.array.Array> shape=[1000, 1024])
        }
    },
    aux2: {
        conv: {
     

IvyException: Containers did not have identical structure:

[34m{[0m
    [32maux4A[0m[35m:[0m [34m{[0m
        [32mconv[0m[35m:[0m [34m{[0m
            [32mbn[0m[35m:[0m [34m{[0m
                [32mb[0m[35m:[0m (<[34mclass[0m ivy.data_classes.array.array.Array> [35mshape=[0m[128]),
                [32mrunning_mean[0m[35m:[0m (<[34mclass[0m ivy.data_classes.array.array.Array> [35mshape=[0m[128]),
                [32mrunning_var[0m[35m:[0m (<[34mclass[0m ivy.data_classes.array.array.Array> [35mshape=[0m[128]),
                [32mw[0m[35m:[0m (<[34mclass[0m ivy.data_classes.array.array.Array> [35mshape=[0m[128])
            [34m}[0m,
            [32mconv[0m[35m:[0m [34m{[0m
                [32mw[0m[35m:[0m (<[34mclass[0m ivy.data_classes.array.array.Array> [35mshape=[0m[1, 1, 512, 128])
            [34m}[0m
        [34m}[0m,
        [32mfc1[0m[35m:[0m [34m{[0m
            [32mb[0m[35m:[0m (<[34mclass[0m 